In [1]:
import numpy as np
import torch
import os
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
print(f"PyTorch version:[{(torch.__version__)}].")

# Device Configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"This notebook use {(device)}")

PyTorch version:[1.7.0+cu101].
This notebook use cuda:0


In [2]:
EPOCHS = 10
BATCH_SIZE = 16
LEARNING_RATE = 0.001

In [3]:
from torchvision import datasets, transforms, models

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229,0.224,0.225])])

test_transfomrs = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

DATASET_PATH = '/content/drive/MyDrive/Colab Notebooks/ball_dataset'

train_data = datasets.ImageFolder(DATASET_PATH + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(DATASET_PATH + '/test', transform=test_transfomrs)

train_iter = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
test_iter = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)

print("Preparing dataset done!")

Preparing dataset done!


# Define Model

In [4]:
class Model(nn.Module):
    def __init__(self, input_size=[3,224,224],
                 hidden_size=[32,64],
                 num_classes=7, init_weight="he", init_bias="zero"):
        super(Model, self).__init__()

        self.init_weight = init_weight
        self.init_bias = init_bias

        layer_list = []
        prev_channel = input_size[0]

        for idx in range(len(hidden_size)):
            layer_list.append(nn.Conv2d(prev_channel, hidden_size[idx], kernel_size=3, stride=1, padding=1))
            layer_list.append(nn.BatchNorm2d(hidden_size[idx]))
            layer_list.append(nn.ReLU(True))
            layer_list.append(nn.MaxPool2d(kernel_size=2, stride=2))
            prev_channel = hidden_size[idx]

        layer_list.append(nn.Flatten())
        feature_size = int(input_size[1]/ 2**len(hidden_size))
        layer_list.append(nn.Linear(feature_size*feature_size*prev_channel, 7))

        self.net = nn.Sequential(*layer_list)

        self.init_params()
    
    
    def init_params(self):

        init_weight_method = {
            "he": nn.init.kaiming_normal_,
            "xavier":nn.init.xavier_normal_
        }
        assert self.init_weight in init_weight_method.keys(), f'Select the weight initialization method in {list(init_weight_method.keys())}'

        init_bias_method = {
            "zero":nn.init.zeros_,
            "uniform":nn.init.uniform_
        }
        assert self.init_bias in init_bias_method.keys(), f'Select the bias initialization method in {list(init_bias_method.keys())}'


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init_weight_method[self.init_weight](m.weight)
                init_bias_method[self.init_bias](m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, X):
        return self.net(X)


In [5]:
model = Model(hidden_size=[64,32,64]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [6]:
total_params = 0
for param_name, param in model.named_parameters():
    if param.requires_grad:
        total_params += len(param.reshape(-1))
print(f'Number of Total Parameters: {total_params:,d}')

Number of Total Parameters: 390,311


# Define help function

In [7]:
def test_eval(model, data_iter, batch_size):
    with torch.no_grad():
        test_loss = 0
        total = 0
        correct = 0
        model.eval()
        for batch_img, batch_lab in data_iter:
            X = batch_img.view(-1, 3, 224, 224).to(device)
            Y = batch_lab.to(device)
            y_pred = model(X)
            _, predicted = torch.max(y_pred.data, 1)
            correct += (predicted == Y).sum().item()
            total += batch_img.size(0)
        val_acc = (100 * correct / total)
        model.train()
    return val_acc

In [ ]:
# Training Phase
print_every = 1
best_accuracy = 0
print("Strart training !")
checkpoint_dir = "/content/drive/MyDrive/Colab Notebooks/ball_dataset/weights"

if os.path.exists(checkpoint_dir):
    model = torch.load(f'{checkpoint_dir}/model.pt')
    model.load_state_dict(
        torch.load(f'{checkpoint_dir}/all.pt'))
    checkpoint = torch.load(f'{checkpoint_dir}/all.tar')
    # model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])

else:
    
